In [2]:
import re
import html
import morfeusz2

import numpy as np

from tqdm.auto import tqdm
from time import sleep

In [2]:
with open("../data/cytaty.txt", "r") as f:
    text = f.read()

In [3]:
def preprocess_quotes(quotes, size = 100000):
    quotes = re.split('[*.!?]', quotes)
    quotes = [html.unescape(sentence.strip()) for sentence in quotes]
    quotes = [sentence for sentence in quotes if len(sentence) > 12]
    return np.array(quotes)[:size]

In [4]:
def lematize_quote(quote, morph):
        analysis = morph.analyse(quote)
        current_idx = 0
        current_stemmed_quote = []
        for i, j, interp in analysis:
            if i == current_idx:
                current_stemmed_quote.append(interp[1].partition(":")[0])
                current_idx += 1
        current_stemmed_quote = [word for word in current_stemmed_quote if word.isalpha()]
        current_stemmed_quote = [word for word in current_stemmed_quote if len(word) > 3]
        return set(current_stemmed_quote)

In [5]:
def lematize_quotes(quotes, morph):
    stemmed_quotes = []
    for quote in tqdm(quotes):
        stemmed_quotes.append(lematize_quote(quote, morph))
    # unique_stemmed_quotes, unique_idx = np.unique(stemmed_quotes, return_index=True)
    # return unique_stemmed_quotes, quotes[unique_idx]
    return stemmed_quotes

In [6]:
def score_quotes(quotes, target):
    scores = []
    for quote in quotes:
        score = (len(quote & target))/(0.2*len(quote) + len(target))
        scores.append(score)
    return np.array(scores)

In [7]:
def prepare_response(scores, quotes, randomize=True):
    idx = scores.argsort()[-10:]
    top10 = quotes[idx]
    top10_scores = scores[idx]**2
    top10_scores = top10_scores/top10_scores.sum()
    if top10_scores[-1] < 0.1:
        return "Postępuj tak, abyś długo żył i dobrze ci się powodziło."
    if not randomize:
        choice = 9
    else:
        choice = np.random.choice(range(10), 1, p=top10_scores)[0]
    return top10[choice], idx[choice]

In [3]:
morph = morfeusz2.Morfeusz()

In [8]:
quotes = preprocess_quotes(text)
lematized_quotes = lematize_quotes(quotes, morph)

  0%|          | 0/100000 [00:00<?, ?it/s]

replacing invalid utf8 sequence with replacement character
replacing invalid utf8 sequence with replacement character
replacing invalid utf8 sequence with replacement character


In [9]:
prompts = []
while True:
    current_prompt = input("JA: ")
    print(f"JA: {current_prompt}")
    if current_prompt == "Koniec":
        print("MCR: Cześć!")
        break
    prompts.append(current_prompt)
    lematized_current_prompt = lematize_quote(current_prompt, morph)
    scores = score_quotes(lematized_quotes, lematized_current_prompt)
    response, idx = prepare_response(scores, quotes)
    print(f"MCR: {response}")
    np.delete(lematized_quotes, idx)
    np.delete(quotes, idx)
    sleep(1)

JA: Byłem wczoraj w kinie.
MCR: Inaczej jest w kinie
JA: ale formę europejską mają nasi sportowcy!
MCR: Te hurysy anielskie mają dusze, to te właśnie Byron kochał po europejsku
JA: ostatnie lata w historii polski były raczej udane
MCR: Także polscy inteligenci w większości mają w swej historii rodzinnej gwałtowny awans społeczny zazwyczaj w pierwszych latach PRL
JA: czytałem wesele Wyspiańskiego, ale wolę wesele Smarzowskiego
MCR: Czytam Czechowa
JA: czyli się zgadzamy
MCR: <br />'''Haruka:''' Zgadza się
JA: jesteś pesymistą
MCR: Czasem mamy pesymistów, którzy nam mówią: „Żegnamy dziś ostatniego prawdziwego Polaka”
JA: a czym dla ciebie jest szczęście?
MCR: Co to jest szczęście
JA: czyli wracamy do sportu. jaki sport uprawiasz?
MCR: Byłem świadkiem zdzierstwa, jakie uprawiano po parafiach
JA: przecież w ten sposób niszczysz swoje życie!
MCR: Ochroni się jedno życie, a potem trzeba reszt swego strawić na walce, by to uratowane nie niszczyło życia innych
JA: a siebie samego rozumiesz?
MC